## 딥러닝과 OpenCV를 활용해 사진 속 글자 검출하기

이 글에서는 이미지 검수를 기계적으로 처리할 수 있게 딥러닝과 이미지 처리 라이브러리인 OpenCV를 활용해 이미지에 포함된 글자를 검출하는 방법을 살펴보겠습니다.  

이미지에 포함된 글자를 검출하는 방법은 개념상 R-CNN(Regions with CNN features)과 유사하지만 이 글에서는 문자 유형 검출의 효율성과 정확성을 보장하기 위해 OpenCV를 활용한 이미지 전처리(pre-processing) 과정에 초점을 맞추어 설명하겠습니다. 이미지 인식과 학습에는 CNN(convolutional neural network)과 Inception-v3 모델을 활용했습니다.

다음과 같은 이미지 데이터를 CNN으로 학습시키면 당연히 '책상'으로 분류한다. CNN은 입력 이미지를 대표하는 특징(feature)을 기반으로 학습하기 때문에 글자인 www.naver.com은 검출되지 않는다.

![이미지](https://d2.naver.com/content/images/2017/09/helloworld-201709-OpenCV-01.png)

화면을 임의로 분할하거나 일정 비율로 분할해 CNN에 인식시키는 방법도 있겠지만 텍스트가 있다고 추정되는 영역만 바로 추출해서 CNN에 인식시키는 것이 더 효율적인 방법이라고 생각된다. 그래서 이 문제의 해결을 '글자 영역만 떼어 내서 학습하고 분류할 수 있을까?'라는 질문에서 시작했다.


### 문자 특징만 잡아낼 수 있을까
이미지에서 글자를 찾는 것은 엄밀히 보면 문자의 특징만 특정해 추출한다기보다는 주어진 이미지에서 색깔(color)을 기반으로 그룹을 지은 덩어리를 추출하는 것이라고 볼 수 있다. 이미지 자체에서 윤곽선을 위주로 한 그룹을 추출해 낼 수 있다면 텍스트 덩어리도 손쉽게 찾을 수 있을 것이다.


### 경계 그룹 찾기
Contour는 이미지에서 연속된 점(point)과 색깔을 분석한 등고선 형태의 경계 그룹이다. 이미지 데이터에서 사물의 형태(shape)를 탐지하고 사물(object)을 탐지할 때 유용하게 사용된다. 자동차의 번호판 인식 등에도 많이 사용된다.

OpenCV는 Contour를 찾아 주는 findContours() 메서드를 제공한다.

> Contour의 개념과 findContours() 메서드 사용 방법에 관해서는 "Image Contours"를 참고한다.

![이미지](https://d2.naver.com/content/images/2017/09/helloworld-201709-OpenCV-02.png)
                            <Contour 추출 예>
                            
이미지에 글자가 있을 때는 주변과 색깔이 다르거나 연속된 점의 구조가 다르기 때문에 글자를 탐지하기 쉽다. 마치 검은 배경에서 하얀색 덩어리들을 찾아 묶는 것과 같다. 그래서 글자를 탐지할 때는 이미지를 회색조(gray scale) 이미지나 흑백 이미지(binary image)로 변환한 다음 Contour를 추출한다.

다음은 이미지를 변환해서 Contour를 추출하는 과정의 예다.
![이미지](https://d2.naver.com/content/images/2017/09/helloworld-201709-OpenCV-03.png)
<Contour 추출 과정>

흑백 이미지를 얻을 때는 회색조로 변환한 Gray Scale 이미지에 임계(threshold)를 적용한다. 즉, 특정값인 임계값 이하의 값을 갖는 픽셀을 검정색으로 변환하고, 임계값 이상의 값을 갖는 픽셀을 흰색으로 변환해 흑백 이미지를 얻는다.

Global Threshold 이미지는 고정값인 127을 임계값으로 설정하고 픽셀을 변환해 얻은 흑백 이미지다. Adaptive Threshold 이미지는 영역을 분할하고 임계값을 자동으로 조정해 얻은 흑백 이미지다. 이 흑백 이미지에서 사각형 영역을 추출해 원본 이미지에 매핑한 결과가 Contour Results 1 이미지와 Contour Results 2 이미지다.

예를 보면 알 수 있듯이 영역을 분할해 임계값을 적용하면 마룻바닥이나 불필요한 영역 등 잡영(noise)을 제거하는 효과가 있기 때문에 사물을 탐지할 때 효과적으로 Contour를 추출할 수 있다.

임계에 대해 조금만 더 살펴보겠다.

임계는 Global Threshold와 Adaptive Threshold로 나눌 수 있다. Global Threshold를 적용하면 이미지 전체에 임계가 적용돼 다음 예의 Global 이미지에서처럼 버려지는 영역이 많아진다. 하지만 Adaptive Threshold는 화면을 적절히 분할하고 각 영역에 적합한 임계를 적용하기 때문에 다음 예의 Mean 이미지와 Gaussian 이미지처럼 좀 더 좋은 결과가 나온다.

![이미지](https://d2.naver.com/content/images/2017/09/helloworld-201709-OpenCV-04.png)
<임계 유형별 예>

그림 3의 Contour Results 1 이미지와 Contour Results 2 이미지에서 초록색 사각형으로 표시된 부분은 Contour의 그룹이다. 아주 작은 그룹까지 찾으면 너무 많은 Contour가 생긴다. 그래서 보통은 사각형 영역의 크기가 어느 정도 이상이거나 밀도가 어느 정도 이상이 아닌 영역은 제외한다. 다음은 너비가 40픽셀보다 크고 높이가 10픽셀보다 큰 영역만 Contour로 추출한 결과다. Contour의 양이 상당히 줄어들고 정교해졌다.

![이미지](https://d2.naver.com/content/images/2017/09/helloworld-201709-OpenCV-05.png)
<40 x 10픽셀 이상인 영역만으로 추출한 Contour>